In [5]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from collections import Counter
from num2words import num2words

import nltk
import os
import string
import numpy as np
import copy
import pandas as pd
import pickle
import re
import math




In [6]:
title = "stories"
alpha = 0.3

In [7]:
# file = "I wanted the gold, and I sought it, I scrabbled and mucked like a slave."
file = open("index.html", 'r')
text = file.read().strip()
file.close()
text

'<HTML>\n<TITLE>T E X T F I L E S</TITLE>\n<BODY BGCOLOR="#000000" TEXT="#00FF00" LINK="#00FF00" ALINK="#00AA00" VLINK="#00AA00">\n<H1>Stories: SRE: The Solar Realms Elite Stories</H1>\n<P>\nJosh Renaud writes"It\'s been 10 years since I first wrote these stories and uploaded them to the file sections of several St. Louis BBSes. I was about 13 or 14 years old at the time.  The first three stories (SRE I-III) were narrative descriptions of an actual game of Solar Realms Elite played on the Knight\'s Armor BBS. It really was a great game, though these early stories didn\'t do it much justice.  People did read and download the stories and a few of them encouraged me to write more. So I did. The subsequent stories came solely from my imagination, except "Feqh Galaxy" and "Galaxy Sei," which were both based on games I participated in."\n<P>\n"Since I was a kid, my writing had a lot of room for improvement. Equally funny as the writing itself is the massive imaginary fan base I addressed in 

In [8]:
file_name = re.findall('><A HREF="(.*)">', text)
file_title = re.findall('<BR><TD> (.*)\n', text)
# file_name = file_name[2:]
file_name

# for j in range(len(file_name)):
#         dataset.append((str(i) +"/"+ str(file_name[j]), file_title[j]))


['sre01.txt',
 'sre02.txt',
 'sre03.txt',
 'sre04.txt',
 'sre05.txt',
 'sre06.txt',
 'sre07.txt',
 'sre08.txt',
 'sre09.txt',
 'sre10.txt',
 'sre_feqh.txt',
 'sre_finl.txt',
 'sre_sei.txt',
 'sretrade.txt',
 'srex.txt']

In [9]:
file_title
print(len(file_name), len(file_title))

15 15


In [10]:

dataset = []
for j in range(len(file_name)):
        dataset.append((str(file_name[j]), file_title[j]))
dataset

[('sre01.txt',
  'SRE: The Saga Of The Best SRE Game Ever Played! By Josh Renaud'),
 ('sre02.txt',
  'Solar Realms Elite: The True Story of the Unsung Heroes, by Josh Renaud'),
 ('sre03.txt', "Solar Realms Elite: Ultra's Untold Story by Josh Renaud"),
 ('sre04.txt', 'Solar Realms Elite IV: The Confrontation, by Josh Renaud'),
 ('sre05.txt', 'Solar Realms Elite V: The Underground, by Josh Renaud'),
 ('sre06.txt', 'Solar Realms Elite VI: The Alliance Restored, by Josh Renaud'),
 ('sre07.txt', 'Solar Realms Elite 7: Petros, by Josh Renaud'),
 ('sre08.txt', 'Solar Realms Elite VIII: Kazik, by Josh Renaud'),
 ('sre09.txt',
  'Solar Realms Elite IX: Survival of the Fittest, by Josh Renaud'),
 ('sre10.txt', 'Solar Realms Elite X: Legacies, by Josh Renaud'),
 ('sre_feqh.txt', 'Solar Realms Elite: The Feqh Galaxy, by Josh Renaud'),
 ('sre_finl.txt', 'Solar Realms Elite: The Finale by Josh Renaud'),
 ('sre_sei.txt', 'Solar Realms Elite: Galaxy Sei, by Josh Renaud'),
 ('sretrade.txt', 'The SRE Co

In [11]:
len(dataset)
N = len (dataset)

In [12]:
def convert_lower_case(data):
    return np.char.lower(data)


In [13]:
def remove_stop_words(data):
    stop_words = stopwords.words('english')
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words and len(w) > 1:
            new_text = new_text + " " + w
    return new_text


In [14]:
def remove_apostrophe(data):
    return np.char.replace(data, "'", "")

In [15]:
def stemming(data):
    stemmer= PorterStemmer()
    
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + stemmer.stem(w)
    return new_text

In [16]:

def convert_numbers(data):
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        try:
            w = num2words(int(w))
        except:
            a = 0
        new_text = new_text + " " + w
    new_text = np.char.replace(new_text, "-", " ")
    return new_text


In [17]:
def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', '')
    return data


In [18]:
def preprocess(data):
    data = convert_lower_case(data)
    data = remove_punctuation(data) #remove comma seperately
    data = remove_apostrophe(data)
    data = remove_stop_words(data)
    data = convert_numbers(data)
    data = stemming(data)
    data = remove_punctuation(data)
    data = convert_numbers(data)
    data = stemming(data) #needed again as we need to stem the words
    data = remove_punctuation(data) #needed again as num2word is giving few hypens and commas fourty-one
    data = remove_stop_words(data) #needed again as num2word is giving stop words 101 - one hundred and one
    return data

In [22]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [24]:
processed_text = []
processed_title = []

for i in dataset[:N]:
    file = open(i[0], 'r', encoding="utf8", errors='ignore')
    text = file.read().strip()
    file.close()

    processed_text.append(word_tokenize(str(preprocess(text))))
    processed_title.append(word_tokenize(str(preprocess(i[1]))))
#     print(processed_title)

In [28]:
DF = {}

for i in range(N):
    tokens = processed_text[i]
    for w in tokens:
#         print(w)
        try:
            DF[w].add(i)
        except:
            DF[w] = {i}

    tokens = processed_title[i]
    for w in tokens:
        try:
            DF[w].add(i)
        except:
            DF[w] = {i}
for i in DF:
    print(i)
    print(DF[i])
    #This will appear where the word comes in the documents.
    DF[i] = len(DF[i])

sre
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14}
saga
{0, 4, 14}
best
{0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13}
game
{0, 1, 2, 3, 5, 10, 12, 13, 14}
ever
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14}
play
{0, 1, 2, 3, 8, 12, 13, 14}
josh
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14}
renaud
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14}
bbse
{0, 3}
call
{0, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14}
solar
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14}
realm
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14}
elit
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14}
devolop
{0, 3}
ibm
{0}
version
{0, 10, 11, 13, 14}
atari
{0, 2, 3}
see
{0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14}
realli
{0, 1, 2, 3, 5, 6, 9, 10, 11, 12, 13, 14}
fun
{0, 12, 7}
get
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14}
geniu
{0}
good
{0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14}
evil
{0, 3, 4, 5, 6, 9, 11, 14}
peopl
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14}
driven
{0}
succeed
{0, 11, 3, 12}
becom
{

{2, 3, 4, 5, 7, 9, 12}
rest
{2, 7, 11, 12, 14}
voila
{2}
im
{2, 5, 6, 7, 8, 9, 10, 11, 12, 14}
fail
{2, 3, 4, 5, 10, 12, 14}
diminish
{2}
discov
{2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 14}
complet
{2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14}
overthrow
{2, 3, 4, 6, 9}
obviou
{8, 2, 12, 14}
propaganda
{2}
heed
{2}
cri
{8, 9, 2, 10}
spark
{2, 3, 5, 8, 9, 10, 11, 12}
command
{2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14}
instead
{2, 3, 9, 11, 14}
mistak
{8, 2, 5, 7}
gab
{2}
stupid
{2, 3, 4, 5, 8}
bronc
{2}
argu
{2, 5}
onth
{2}
verg
{9, 2, 4, 6}
penni
{2}
pincher
{2}
fulfil
{2}
bare
{8, 2, 12}
necesseti
{2}
receiv
{2, 3, 5, 14}
funnel
{2}
repair
{8, 2, 10, 6}
empier
{2}
standoff
{2}
either
{2, 3, 4, 6, 12, 14}
positionof
{2}
grab
{2, 5, 7, 10, 12}
giant
{8, 2, 11, 4}
broke
{2, 4, 11, 12, 14}
cash
{2, 3, 4, 12, 13}
flow
{2, 3, 5, 11, 13}
sever
{2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14}
shambl
{8, 2, 3, 7}
longest
{2, 3}
record
{2, 10, 6}
achiev
{2, 13, 6}
goal
{2, 4, 6, 9, 11, 14}
leadership
{2, 5, 7, 8, 10}


{3, 4}
distrubut
{3}
cooridin
{3}
normal
{3, 6, 7, 8, 9, 10, 11, 14}
babi
{3}
boy
{10, 3, 4, 5}
reali
{3}
bioweapon
{3, 4}
heard
{3, 4, 5, 7, 8, 9, 10, 11, 12, 14}
shipment
{3}
chemic
{3, 4, 5, 9, 10, 11, 14}
chemweapon
{3, 4}
lower
{11, 3, 7}
anyway
{3, 5, 7, 8, 14}
atidi
{3}
profit
{3, 13}
intend
{3, 7, 8, 9, 12, 14}
hold
{3, 4, 5, 10, 11, 12, 13, 14}
hurt
{9, 3, 5, 14}
concern
{3, 5, 11, 12, 14}
level
{3, 5, 8, 11, 13}
horrif
{3}
tal
{3}
replac
{10, 3, 4, 5}
se
{10, 3}
skill
{3, 12, 5, 14}
ten
{3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14}
injur
{3}
twenti
{10, 3, 14, 7}
eight
{3, 6, 8, 10, 14}
five
{3, 4, 5, 6, 7, 8, 9, 10, 11, 12}
eleven
{3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14}
hisinsurg
{3}
coup
{3, 5}
low
{9, 3, 12, 13}
afford
{8, 3, 14}
dunerealm
{3}
distant
{3}
memori
{9, 3}
tell
{3, 5, 6, 7, 8, 9, 10, 12, 14}
andattack
{3}
ultrad
{3}
wedg
{3}
situationerupt
{3}
refu
{3, 5, 7, 11, 12, 14}
accept
{11, 3}
realiti
{11, 3}
stubborn
{3}
natur
{3, 6, 9, 11, 12}
wish
{3, 5, 6, 7, 10, 11, 12,

behind
{5, 7, 8, 9, 11, 12, 14}
wast
{5, 7, 8, 11, 12}
miltari
{5}
vulner
{5, 7}
withdraw
{8, 5, 6}
rechristen
{5}
partisan
{5}
lieu
{5}
terrorist
{9, 5, 14}
envoy
{5, 7}
impor
{5}
convent
{10, 5}
uncheck
{5, 7}
legal
{5}
proceed
{11, 5, 6}
suicid
{12, 5}
sicken
{5}
reincarn
{5}
engin
{5, 6, 7, 8, 10, 11, 14}
outwit
{5}
cun
{12, 5, 6}
ploy
{5, 6}
bought
{5}
nioxyl
{10, 5}
missl
{5}
niobomb
{5}
deadli
{10, 5, 14, 7}
combin
{8, 12, 5, 14}
nickel
{5}
acid
{5}
antiox
{11, 5, 7}
oxygen
{10, 5}
antimatt
{5}
arsen
{5}
map
{9, 5, 14}
compound
{5}
trilithium
{5}
psioxid
{5}
dampen
{5}
irrit
{10, 5}
tripsiox
{5}
shield
{14, 5, 6, 7}
headach
{5}
develop
{5, 6, 7, 8, 11, 12, 14}
combat
{11, 5}
discredit
{5}
process
{5, 7, 8, 12, 13}
unfit
{5}
deep
{5, 8, 9, 10, 11, 12, 14}
headquart
{8, 5, 14, 7}
pool
{5}
formid
{5}
front
{5, 7, 8, 9, 10}
christen
{5}
planetari
{12, 5}
pfg
{5}
hierarchi
{13, 5}
respon
{5, 7, 8, 10, 12}
yesterday
{5, 14}
format
{5, 7}
statement
{13, 5}
via
{5, 14}
public
{8, 5, 6}


magazin
{13, 6}
grossworld
{8, 13, 6}
publish
{8, 13, 6}
compani
{6, 7, 9, 11, 13}
spellbind
{7}
text
{7, 8, 9, 11, 12, 13}
suspen
{7}
proven
{7}
theori
{13, 7}
difficult
{7, 9, 11, 12, 13, 14}
fo
{7}
adversari
{7}
buzz
{14, 7}
offer
{11, 12, 7}
successor
{9, 14, 7}
frame
{7}
item
{13, 7}
donat
{8, 7}
alon
{8, 10, 12, 7}
anywh
{8, 7}
reorgan
{7}
refurbish
{7}
manag
{10, 11, 7}
scroung
{7}
compen
{7}
account
{7}
percent
{11, 14, 7}
seclud
{7}
prefer
{7}
brief
{7, 8, 10, 11, 14}
stint
{7}
board
{9, 11, 14, 7}
exactli
{9, 10, 14, 7}
unsur
{12, 14, 7}
awar
{14, 7}
structur
{11, 13, 14, 7}
institut
{11, 14, 7}
evenut
{13, 7}
sovereign
{11, 7}
insight
{7}
lab
{7, 9, 10, 11, 14}
meticul
{9, 7}
correct
{10, 7}
phase
{7}
gluon
{8, 7}
deattach
{7}
subatom
{7}
ecstat
{8, 9, 14, 7}
herald
{7}
unpredict
{7}
led
{8, 12, 7}
vast
{7, 8, 10, 11, 12, 13, 14}
unconvent
{7}
strangest
{7}
uea
{7, 8, 9, 11, 14}
reappear
{8, 7}
propser
{7}
theyv
{12, 13, 14, 7}
imminenec
{7}
shift
{11, 14, 7}
lean
{14, 7}
le

{9, 14}
fashion
{9}
weep
{9}
rock
{9}
dearest
{9}
goodbi
{9, 11}
adverti
{10}
repeatedli
{10}
feqhican
{10, 11, 14}
jirca
{10, 11}
phee
{10, 11}
reign
{10}
pervert
{10}
sickest
{10}
whichev
{10}
reward
{10}
throneba
{10}
throne
{10}
dread
{10}
marri
{10}
segment
{10}
typic
{10, 11}
soldier
{10, 13}
societi
{10, 11}
entertain
{10}
buctia
{10}
repress
{10}
siberian
{10}
intrigu
{10}
creativ
{10}
winner
{10}
humbl
{10}
iseylon
{10, 11, 14}
ureya
{10}
frezar
{10}
rufreji
{10}
per
{10, 12}
qualif
{10}
laka
{10, 11, 14}
dina
{10}
minist
{10, 11}
honawa
{10}
underchief
{10}
staff
{10, 14}
opert
{10}
clandenstin
{10, 11}
toll
{10}
superb
{10}
ancient
{10}
wasalia
{10}
refreji
{10}
advanc
{10, 13, 14}
line
{10, 12, 14}
sabotag
{10}
earli
{10, 12}
train
{10, 11}
camp
{10}
unprotect
{10}
vibrat
{10}
viewport
{10}
broken
{10, 12, 14}
port
{10, 11, 14}
air
{10, 14}
empti
{10}
describ
{10}
oversight
{10}
triumph
{10}
consort
{10}
randomli
{10}
thrust
{10}
hmm
{10, 14}
halt
{10, 11}
refregi
{10}
unde

{14}
straw
{14}
inquiri
{14}
night
{14}
access
{14}
thin
{14}
complex
{14}
sought
{14}
supri
{14}
urg
{14}
syneth
{14}
dissappear
{14}
spnding
{14}
ver
{14}
criminolog
{14}
cheek
{14}
tall
{14}
block
{14}
rapidli
{14}
tonight
{14}
cast
{14}
buffer
{14}
compat
{14}
cataclysm
{14}
video
{14}
footag
{14}
synth
{14}
configur
{14}
updat
{14}
enclo
{14}
shoot
{14}
greatest
{14}
creap
{14}
fist
{14}
march
{14}
panic
{14}
patch
{14}
overlay
{14}
theft
{14}
replic
{14}
pale
{14}
comparison
{14}
veng
{14}
loo
{14}
topic
{14}
neurolog
{14}
disord
{14}
oral
{14}
psyodid
{14}
inhal
{14}
offlin
{14}
reclin
{14}
rendezv
{14}
growl
{14}
eta
{14}
royalti
{14}
comlink
{14}
cordial
{14}
cavox
{14}
vocal
{14}
alik
{14}
avayo
{14}
commend
{14}
arisen
{14}
derang
{14}
herebi
{14}
tarheil
{14}
compet
{14}
expert
{14}
experti
{14}
ursurp
{14}
commonwealth
{14}
grandson
{14}
stride
{14}
commot
{14}
proof
{14}
assumn
{14}
passeng
{14}
cubicl
{14}
acquaint
{14}
instabl
{14}
section
{14}
weight
{14}
contest
{14}


In [37]:
total_vocab_size = len(DF)
total_vocab_size

3962

In [38]:
total_vocab = [x for x in DF]
print(total_vocab)

['sre', 'saga', 'best', 'game', 'ever', 'play', 'josh', 'renaud', 'bbse', 'call', 'solar', 'realm', 'elit', 'devolop', 'ibm', 'version', 'atari', 'see', 'realli', 'fun', 'get', 'geniu', 'good', 'evil', 'peopl', 'driven', 'succeed', 'becom', 'ultraempir', 'youll', 'happen', 'stori', 'want', 'let', 'know', 'inform', 'pick', 'true', 'may', 'exact', 'onto', 'long', 'ago', 'galaxi', 'born', 'moder', 'galactiv', 'coordintor', 'rule', 'race', 'popul', 'made', 'came', 'one', 'start', 'littl', 'theyt', 'becam', 'big', 'die', 'empir', 'name', 'ultra', 'experi', 'help', 'seed', 'easili', 'conquer', 'whole', 'ala', 'thing', 'chang', 'ruler', 'left', 'visit', 'didnt', 'return', 'week', 'maceland', 'intent', 'back', 'find', 'ruin', 'somehow', 'thought', 'must', 'resurrect', 'wage', 'war', 'daili', 'attack', 'witfh', 'lose', 'ground', 'slowli', 'admit', 'defeat', 'truce', 'mace', 'content', 'peac', 'idea', 'allianc', 'tri', 'coupl', 'join', 'sadden', 'never', 'got', 'grow', 'soon', 'ask', 'three', 'w

In [39]:
def doc_freq(word):
    c = 0
    try:
        c = DF[word]
    except:
        pass
    return c

In [57]:
doc = 0

tf_idf = {}

for i in range(N):
    
    tokens = processed_text[i]
    
    counter = Counter(tokens + processed_title[i])
    #Counter will return the map with count of that word appearing in that document.
    words_count = len(tokens + processed_title[i])
    
    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df = doc_freq(token)
#         df=no. of documents containing this word, in all documents how many this word is accuring
        idf = np.log((N+1)/(df+1))
        
        tf_idf[doc, token] = tf*idf

    doc += 1
print (doc)

15


In [51]:
doc = 0

tf_idf_title = {}

for i in range(N):
    
    tokens = processed_title[i]
    counter = Counter(tokens + processed_text[i])
    words_count = len(tokens + processed_text[i])

    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = np.log((N+1)/(df+1)) #numerator is added 1 to avoid negative values
        
        tf_idf_title[doc, token] = tf*idf

    doc += 1

In [52]:
for i in tf_idf:
    tf_idf[i] *= alpha

for i in tf_idf_title:
    tf_idf[i] = tf_idf_title[i]

In [53]:
len(tf_idf)

11852

In [55]:
def matching_score(k, query):
    preprocessed_query = preprocess(query)
    tokens = word_tokenize(str(preprocessed_query))

    print("Matching Score")
    print("\nQuery:", query)
    print("")
    print(tokens)
    
    query_weights = {}

    for key in tf_idf:
        
        if key[1] in tokens:
            try:
                query_weights[key[0]] += tf_idf[key]
            except:
                query_weights[key[0]] = tf_idf[key]
    
    query_weights = sorted(query_weights.items(), key=lambda x: x[1], reverse=True)

    print(query_weights)
    
    l = []
    
    for i in query_weights[:10]:
        l.append(i[0])
    
    print(l)
    

matching_score(10, "Without the drive of Rebeccah's insistence, Kate lost her momentum. She stood next a slatted oak bench, canisters still clutched, surveying")

Matching Score

Query: Without the drive of Rebeccah's insistence, Kate lost her momentum. She stood next a slatted oak bench, canisters still clutched, surveying

['without', 'drive', 'rebeccah', 'insist', 'kate', 'lost', 'momentum', 'stood', 'next', 'slat', 'oak', 'bench', 'canist', 'still', 'clutch', 'survey']
[(11, 0.0011398426080150177), (4, 0.0009620374367012203), (2, 0.0006726405324491299), (14, 0.0006662744500489751), (1, 0.0006443389719697403), (6, 0.0006440604934574291), (9, 0.000596803681450333), (3, 0.00042902117401386747), (8, 0.0004220368150114093), (12, 0.00040124527592417055), (13, 0.0003701485421405042), (7, 0.00027153926003823265), (0, 0.00022973723620816575), (10, 0.00022882598695889045), (5, 0.00017609775213734796)]
[11, 4, 2, 14, 1, 6, 9, 3, 8, 12]
